### 0. 基础环境

In [1]:
# 环境变量
from dotenv import load_dotenv
load_dotenv(dotenv_path=".baidu_env")
# LLM
from langchain_community.llms.baidu_qianfan_endpoint import QianfanLLMEndpoint
# 千帆大模型
from langchain_community.chat_models.baidu_qianfan_endpoint import QianfanChatEndpoint
# 嵌入模型
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
# LLM
llm = QianfanLLMEndpoint(model="ERNIE-4.0-8K")
# 模型构建
chat = QianfanChatEndpoint(model="ERNIE-4.0-8K")

embed = QianfanEmbeddingsEndpoint(model='bge-large-zh')

### 1.  基础检索器

In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

D:\software\anaconda3\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# 实例化在线资源加载
web_loader = WebBaseLoader(web_path="https://baike.baidu.com/item/%E6%AF%9B%E6%B3%BD%E4%B8%9C")

In [4]:
# 加载资源
web_docs = web_loader.load()

In [5]:
# 初始化 spliter
spliter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)

In [6]:
# 切分文档
docs = spliter.split_documents(documents=web_docs)

In [7]:
db = Chroma.from_documents(documents=docs, embedding=embed, persist_directory="./chroma_data3")

[INFO][2024-08-29 21:34:31.094] oauth.py:228 [t:8352]: trying to refresh access_token for ak `i4ba2u***`
[INFO][2024-08-29 21:34:31.320] oauth.py:243 [t:8352]: sucessfully refresh access_token


### 基本检索

In [8]:
# 注意传参数 带阈值参数
retriever = db.as_retriever(
    # 距离度量标准，similarity 默认：欧式距离
    search_type ="similarity_score_threshold",
    # 搜索参数
    search_kwargs={"k": 4, "score_threshold": 0.50}
)

In [9]:
results = retriever.invoke("毛主席是谁？")

### 集成检索器
- 思想：类似与集成学习
- 首先：使用多个检索器对向量数据库进行内容检索
- 然后：基于倒数排序融合算法(RRF)对结果进行融合和排序

In [10]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [12]:
! pip install rank_bm25

In [14]:
# 初始化一个 BM25 检索器
bm25_retriever = BM25Retriever.from_documents(documents=docs)
bm25_retriever.k = 2

In [15]:
# 1. bm25检索
result1 = bm25_retriever.invoke(input="毛主席是谁？")
len(result1)

2

In [16]:
# 2. 基础检索器
result2 = retriever.invoke(input="毛主席是谁？")
len(result2)

4

In [17]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever], 
                                       weights=[0.2, 0.8])

In [18]:
# 3. 集成检索
results = ensemble_retriever.invoke(input="毛主席是谁？")
len(results)

6

In [52]:
# results

In [51]:
# 最终结果展现
for result in results:
    print(result.page_content, "\n\n\n")

毛泽东_百度百科 网页新闻贴吧知道网盘图片视频地图文库资讯采购百科百度首页登录注册进入词条全站搜索帮助首页秒懂百科特色百科知识专题加入百科百科团队权威合作个人中心收藏查看我的收藏0有用+10毛泽东[máo zé dōng]播报上传视频伟大的无产阶级革命家毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝）， [33]笔名子任。湖南湘潭人。中国人民的领袖，伟大的马克思主义者，伟大的无产阶级革命家 [23]、战略家、理论家，中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人，马克思主义中国化的伟大开拓者，近代以来中国伟大的爱国者和民族英雄，中国共产党第一代中央领导集体的核心，领导中国人民彻底改变自己命运和国家面貌的一代伟人 [16] [23]。1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。 



[16] [23]。1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。 [34-35]因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响100人之一。中文名毛泽东别    名石三伢子、二十八画生 等国    籍中国民    族汉族出生地湖南省长沙府湘潭县韶山冲出生日期1893年12月26日逝世日期1976年09月09日毕业院校湖南省立第一师范学校代表作品矛盾论、实践论、论持久战、沁园春·雪主要成就领导夺取中国新民主主义革命的胜利在中国确立社会主义基本制度 [23]毛泽东思想的主要创立者信    仰马克思主义目录1人物生平2大事年表▪1893-1910年▪1911-1920年▪1921-1930年▪1931-1940年▪1941-1950年▪1951-1960年▪1961-1970年▪1971-1976年3主要作品4历史贡献5人物评价人物生平播报编辑毛泽东，1893年12月26日生于湖南韶山 



[1]毛泽东像集（一）大事年表播报编辑1893-1910年1893年12月26日，诞生在湖南省湘潭县韶山冲一个农民家庭。1902年-1909年在家乡韶山六所私塾

### 父文档检索
- 检索时：向量化处理，希望文档小一些，可以更精准的比较语义（细粒度）
- 生成时：上下文参考，希望文档大一些，保留更多的参考上下文（粗粒度）
- 折中办法：
    - 一个文档，先分个几个大块
    - 每个大块，再拆分为几个小块
    - 向量化比较的是小块，最后返回的是大块

In [20]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [21]:
# 父切分器(大段切分器)
parent_spliter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

In [22]:
# 子切分器(小段切分器)
child_spliter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)

In [23]:
# 存储小块的向量库 （没有指定持久化地址，那么默认存储在内存中）
child_chroma = Chroma(embedding_function=embed)

In [24]:
# 创建一个存储大块的内存型存储器（类似于Redis）
parent_store = InMemoryStore()

In [25]:
final_retriever = ParentDocumentRetriever(vectorstore=child_chroma, 
                        docstore=parent_store,
                        child_splitter=child_spliter,
                        parent_splitter=parent_spliter)

In [26]:
final_retriever.add_documents(documents=web_docs)

In [27]:
results = final_retriever.invoke(input="毛主席是谁？")

In [28]:
len(results)

2

In [29]:
for result in results:
    print(len(result.page_content))

1808
2000


In [75]:
child_results = child_chroma.similarity_search_with_relevance_scores(query="毛主席是谁？")

In [76]:
len(child_results)

4

In [81]:
parent_ids = set()
for child_result in child_results:
    print(child_result[0].metadata["doc_id"], "\n\n\n")
    parent_ids.add(child_result[0].metadata["doc_id"])

74e3f568-52f3-4cdf-a2c6-a911f7af4c84 



74e3f568-52f3-4cdf-a2c6-a911f7af4c84 



74e3f568-52f3-4cdf-a2c6-a911f7af4c84 



3f7a75ef-2d94-4ac0-be22-e55a29665d51 





In [82]:
parent_ids

{'3f7a75ef-2d94-4ac0-be22-e55a29665d51',
 '74e3f568-52f3-4cdf-a2c6-a911f7af4c84'}

In [84]:
for doc in parent_store.mget(keys=list(parent_ids)):
    print(len(doc.page_content))

2000
1763


### 多向量检索
- 通过摘要检索文档

In [30]:
from langchain.retrievers import MultiVectorRetriever

In [31]:
# 大块切分
text_spliter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

In [32]:
# 执行大块切分操作
docs = text_spliter.split_documents(documents=web_docs)
len(docs)

17

In [33]:
# 自定定义一个文档摘要 Prompt
from langchain_core.prompts import PromptTemplate

In [34]:
# 自定义一个文档总结 Prompt
summary_prompt = PromptTemplate.from_template(template="请使用200个以内的字来归纳总结下面的文档！\n\n文档为: {doc}")

In [35]:
summary_prompt

PromptTemplate(input_variables=['doc'], template='请使用200个以内的字来归纳总结下面的文档！\n\n文档为: {doc}')

In [111]:
# 把 文档进行总结的 链 chain
chain = (
    {"doc": lambda x : x.page_content}
    | summary_prompt 
    | llm
)

In [112]:
summaries = chain.batch(inputs=docs,config={"max_concurrency": 5})

In [113]:
for summary in summaries:
    print(len(summary))

91
107
106
137
523
291
81
142
480
156
151
100
523
115
84
91
71


In [114]:
for summary in summaries:
    print(summary, "\n\n")

毛泽东是中国人民的领袖，伟大的无产阶级革命家，中国共产党第一代领导集体核心，领导中国人民改变自己命运和国家面貌的一代伟人，他对马列主义、军事及共产党理论有重大贡献，被誉为毛泽东思想。 


毛泽东是中国共产党的核心领导人，他在各个历史时期都有重大贡献。他参与了中共的重要会议和行动，对抗日战争、解放战争等中国近现代史的重要事件有深远影响。他的思想和理论被确立为中共的指导思想，对中国的发展产生了巨大影响。 


该文档概述了毛泽东从建国初期到逝世的主要政治活动和历史事件，包括开国大典、抗美援朝、土地改革、主持制定宪法、提出过渡时期总路线、探索社会主义建设道路、反右倾斗争、主持庐山会议、领导国民经济调整、发动文化大革命等。 


该文主要讲述了从1911年到1930年毛泽东的革命历程和思想转变过程。他在求学期间开始接触革命思想，并积极参与革命活动，逐渐成长为一名坚定的马克思主义者和无产阶级革命家。在长沙，他参与了多次革命实践活动，推动了湖南的革命运动的发展，并为中国共产党的创立和发展做出了重要贡献。 


毛泽东是中国人民的领袖，伟大的马克思主义者，伟大的无产阶级革命家、战略家、理论家，中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人，马克思主义中国化的伟大开拓者，近代以来中国伟大的爱国者和民族英雄，中国共产党第一代中央领导集体的核心，领导中国人民彻底改变自己命运和国家面貌的一代伟人。
毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响100人之一。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。
毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响100人之一。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响100人之一。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。 


毛泽东在中共临时中央全面推行“进

In [115]:
# 存摘要的，向量化存储
vectorstore = Chroma(collection_name="summaries", 
                    embedding_function=embed)

In [117]:
# 存大段文本的，原始存储
store = InMemoryStore()

In [118]:
id_key = "doc_id"

In [119]:
mv_retirever = MultiVectorRetriever(vectorstore=vectorstore, 
                                    docstore=store,
                                    id_key=id_key,
                                    search_kwargs={"k": 4})

In [120]:
import uuid

In [121]:
# 批量生成 文档 id
doc_ids = [str(uuid.uuid1()) for _ in docs]

In [125]:
doc_ids

['97af8f64-562c-11ef-8590-6c02e044efe2',
 '97af8f65-562c-11ef-8e62-6c02e044efe2',
 '97af8f66-562c-11ef-b9f9-6c02e044efe2',
 '97af8f67-562c-11ef-b558-6c02e044efe2',
 '97af8f68-562c-11ef-b6fd-6c02e044efe2',
 '97af8f69-562c-11ef-b0de-6c02e044efe2',
 '97af8f6a-562c-11ef-973e-6c02e044efe2',
 '97af8f6b-562c-11ef-bae3-6c02e044efe2',
 '97af8f6c-562c-11ef-8632-6c02e044efe2',
 '97af8f6d-562c-11ef-886f-6c02e044efe2',
 '97af8f6e-562c-11ef-b973-6c02e044efe2',
 '97af8f6f-562c-11ef-a29b-6c02e044efe2',
 '97af8f70-562c-11ef-ac23-6c02e044efe2',
 '97af8f71-562c-11ef-962b-6c02e044efe2',
 '97af8f72-562c-11ef-98e0-6c02e044efe2',
 '97af8f73-562c-11ef-97fc-6c02e044efe2',
 '97af8f74-562c-11ef-a563-6c02e044efe2']

In [126]:
from langchain_core.documents import Document

In [127]:
summary_docs = [Document(page_content=s, metadata={id_key: doc_ids[i]}) for i, s in enumerate(summaries)]

In [128]:
summary_docs

[Document(metadata={'doc_id': '97af8f64-562c-11ef-8590-6c02e044efe2'}, page_content='毛泽东是中国人民的领袖，伟大的无产阶级革命家，中国共产党第一代领导集体核心，领导中国人民改变自己命运和国家面貌的一代伟人，他对马列主义、军事及共产党理论有重大贡献，被誉为毛泽东思想。'),
 Document(metadata={'doc_id': '97af8f65-562c-11ef-8e62-6c02e044efe2'}, page_content='毛泽东是中国共产党的核心领导人，他在各个历史时期都有重大贡献。他参与了中共的重要会议和行动，对抗日战争、解放战争等中国近现代史的重要事件有深远影响。他的思想和理论被确立为中共的指导思想，对中国的发展产生了巨大影响。'),
 Document(metadata={'doc_id': '97af8f66-562c-11ef-b9f9-6c02e044efe2'}, page_content='该文档概述了毛泽东从建国初期到逝世的主要政治活动和历史事件，包括开国大典、抗美援朝、土地改革、主持制定宪法、提出过渡时期总路线、探索社会主义建设道路、反右倾斗争、主持庐山会议、领导国民经济调整、发动文化大革命等。'),
 Document(metadata={'doc_id': '97af8f67-562c-11ef-b558-6c02e044efe2'}, page_content='该文主要讲述了从1911年到1930年毛泽东的革命历程和思想转变过程。他在求学期间开始接触革命思想，并积极参与革命活动，逐渐成长为一名坚定的马克思主义者和无产阶级革命家。在长沙，他参与了多次革命实践活动，推动了湖南的革命运动的发展，并为中国共产党的创立和发展做出了重要贡献。'),
 Document(metadata={'doc_id': '97af8f68-562c-11ef-b6fd-6c02e044efe2'}, page_content='毛泽东是中国人民的领袖，伟大的马克思主义者，伟大的无产阶级革命家、战略家、理论家，中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人，马克思主义中国化的伟大开拓者，近代以来中国伟大的爱国者和民族英雄，中国共产党第一代中央领导

In [129]:
# 将摘要存入向量库
mv_retirever.vectorstore.add_documents(documents=summary_docs)

['e8f8da81-2f45-40eb-b714-2f32a5020f5b',
 '6681794a-558c-4c99-9554-497cf707e9d2',
 '90af89a6-7cc7-4168-86fc-70512b7bb228',
 '5a824a93-cef3-419c-8b81-2e39c4ff3e67',
 'c766f34a-e918-4aeb-bb05-92678fc9a936',
 '8c62d1a9-5dea-4e5d-9213-0dc57226a891',
 '019b60fd-cf98-4b2f-80f7-0253f52411ba',
 '5c6c6240-98dd-4fc1-a49b-e48eca6cd9a2',
 'e075103c-6e94-4ee9-9b71-a58c62a0a057',
 'f13770d4-5335-421c-9aab-169a60ae0374',
 'ad727be5-f96f-4285-84e1-876cfc629f5c',
 '579a40bf-019d-46c3-a975-63d3ee6af09a',
 '830d36e7-a738-4d0b-a6e7-b06808ba0ff9',
 '59c21d5d-99f2-40e6-a0ed-d1ac2c17ae77',
 'cf64d25c-4fc1-42d2-97a7-2e226e1508b8',
 'f20052e4-59f8-4880-b698-61261b915d9f',
 '6c2bb104-078a-4dfe-bf73-13545ce234cf']

In [130]:
# 文本存储
mv_retirever.docstore.mset(list(zip(doc_ids, docs)))

In [132]:
# 进行检索
results= mv_retirever.invoke(input="毛主席是谁？")

In [133]:
for result in results:
    print(len(result.page_content))

1763
1999
262
1688


In [135]:
summary_results = mv_retirever.vectorstore.similarity_search_with_relevance_scores(query="毛主席是谁？")

In [137]:
summary_results

[(Document(metadata={'doc_id': '97af8f64-562c-11ef-8590-6c02e044efe2'}, page_content='毛泽东是中国人民的领袖，伟大的无产阶级革命家，中国共产党第一代领导集体核心，领导中国人民改变自己命运和国家面貌的一代伟人，他对马列主义、军事及共产党理论有重大贡献，被誉为毛泽东思想。'),
  0.6633318170319487),
 (Document(metadata={'doc_id': '97af8f71-562c-11ef-962b-6c02e044efe2'}, page_content='毛泽东同志是伟大的马克思主义革命家、战略家，他为国家富强、民族振兴不懈奋斗，带领中国人民彻底改变自己命运。他开创了马克思主义中国化，创立了毛泽东思想，并用此指导实践，锻造了伟大光荣的中国共产党，为推进中国社会主义事业作出杰出贡献。'),
  0.6453979115220783),
 (Document(metadata={'doc_id': '97af8f73-562c-11ef-97fc-6c02e044efe2'}, page_content='毛泽东同志缔造了忠诚于党、为人民奋斗的人民军队，是保障国家独立和人民幸福的坚强后盾。他展现出的政治远见、革命信念和领导才能等崇高风范，深受全党全国各族人民敬仰，是激励我们前进的动力。'),
  0.6380640227063945),
 (Document(metadata={'doc_id': '97af8f68-562c-11ef-b6fd-6c02e044efe2'}, page_content='毛泽东是中国人民的领袖，伟大的马克思主义者，伟大的无产阶级革命家、战略家、理论家，中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人，马克思主义中国化的伟大开拓者，近代以来中国伟大的爱国者和民族英雄，中国共产党第一代中央领导集体的核心，领导中国人民彻底改变自己命运和国家面貌的一代伟人。\n毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响100人之一。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。\n毛

In [139]:
for sr in summary_results:
    print(sr[0].metadata)

{'doc_id': '97af8f64-562c-11ef-8590-6c02e044efe2'}
{'doc_id': '97af8f71-562c-11ef-962b-6c02e044efe2'}
{'doc_id': '97af8f73-562c-11ef-97fc-6c02e044efe2'}
{'doc_id': '97af8f68-562c-11ef-b6fd-6c02e044efe2'}


In [140]:
for result in results:
    print(result.metadata)

{'source': 'https://baike.baidu.com/item/%E6%AF%9B%E6%B3%BD%E4%B8%9C', 'title': '毛泽东_百度百科', 'description': '毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝），笔名子任。湖南湘潭人。中国人民的领袖，伟大的马克思主义者，伟大的无产阶级革命家、战略家、理论家，中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人，马克思主义中国化的伟大开拓者，近代以来中国伟大的爱国者和民族英雄，中国共产党第一代中央领导集体的核心，领导中国人民彻底改变自己命运和国家面貌的一代伟人。1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响100人之一。', 'language': 'No language found.'}
{'source': 'https://baike.baidu.com/item/%E6%AF%9B%E6%B3%BD%E4%B8%9C', 'title': '毛泽东_百度百科', 'description': '毛泽东（1893年12月26日-1976年9月9日），字润之（原作咏芝，后改润芝），笔名子任。湖南湘潭人。中国人民的领袖，伟大的马克思主义者，伟大的无产阶级革命家、战略家、理论家，中国共产党、中国人民解放军和中华人民共和国的主要缔造者和领导人，马克思主义中国化的伟大开拓者，近代以来中国伟大的爱国者和民族英雄，中国共产党第一代中央领导集体的核心，领导中国人民彻底改变自己命运和国家面貌的一代伟人。1949至1976年，毛泽东担任中华人民共和国最高领导人。他对马克思列宁主义的发展、军事理论的贡献以及对共产党的理论贡献被称为毛泽东思想。因毛泽东担任过的主要职务几乎全部称为主席，所以也被人们尊称为“毛主席”。毛泽东被视为现代世界历史中最重要的人物之一，《时代》杂志也将他评为20世纪最具影响100人之一。', 'language': 'No language found.'}
{'source